In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time
import os
import sys
import copy
import datetime
import random
import math
import warnings
warnings.filterwarnings('ignore')

In [11]:
import torch
from torch import nn
import torch.nn.functional as F
import torchaudio
from torchaudio import models, transforms
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn

In [4]:
import matplotlib
# matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt 

matplotlib.rcParams['lines.linewidth'] = 1
matplotlib.rcParams['lines.markersize'] = 5

In [5]:
import numpy as np
import IPython.display as ipd
from tqdm import tqdm

In [8]:
sys.path.append("/home/geshi/ChaosMining")

from chaosmining.data_utils import ChaosAudioDataset
from chaosmining.utils import check_make_dir
from chaosmining.audio import parse_argument

from captum.attr import IntegratedGradients, Saliency, DeepLift, FeatureAblation, visualization

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
save_flag = False
n_channels = 10

# Noisy Audio Dataset

In [27]:
# Create training and testing split of the data. We do not use validation in this tutorial.
train_set = ChaosAudioDataset('/data/home/geshi/ChaosMining/data/audio/RBFP', "train")
val_set = ChaosAudioDataset('/data/home/geshi/ChaosMining/data/audio/RBFP', "val")

waveform, label, sample_rate = train_set[0]

In [28]:
print("Shape of waveform: {}".format(waveform.size()))
print("Sample rate of waveform: {}".format(sample_rate))
print("class of waveform: {}".format(label))
print("dataset size train {}, val {}".format(len(train_set), len(val_set)))

Shape of waveform: torch.Size([10, 16000])
Sample rate of waveform: 16000
class of waveform: 0
dataset size train 84843, val 9981


In [32]:
labels = sorted(train_set.classes)
print("number of classes", len(labels))

number of classes 35


In [37]:
ipd.Audio(waveform[0].numpy(), rate=sample_rate)

# Generate Synthetic Audio Data

In [38]:
def label_to_index(word):
    # Return the position of the word in labels
    return torch.tensor(labels.index(word))

def index_to_label(index):
    # Return the word corresponding to the index in labels
    # This is the inverse of label_to_index
    return labels[index]

word_start = "yes"
index = label_to_index(word_start)
word_recovered = index_to_label(index)

print(word_start, "-->", index, "-->", word_recovered)

yes --> tensor(33) --> yes


In [55]:
def collate_fn(batch):

    # A data tuple has the form:
    # waveform, sample_rate, label, speaker_id, utterance_number

    tensors, targets = [], []

    # Gather in lists, and encode labels as indices
    for waveform, label, *_ in batch:
        tensors += [waveform]
        targets += [label]

    # Group the list of tensors into a batched tensor
    tensors = torch.stack(tensors)
    targets = torch.tensor(targets)

    return tensors, targets

In [56]:
batch_size = 64

if device == "cuda":
    num_workers = 1
    pin_memory = True
else:
    num_workers = 0
    pin_memory = False

train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=num_workers,
    pin_memory=pin_memory,
)
val_loader = torch.utils.data.DataLoader(
    val_set,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    collate_fn=collate_fn,
    num_workers=num_workers,
    pin_memory=pin_memory,
)

# Define Network

In [57]:
class M5(nn.Module):
    def __init__(self, n_input=1, n_output=35, stride=16, n_channel=32):
        super().__init__()
        self.conv1 = nn.Conv1d(n_input, n_channel, kernel_size=80, stride=stride)
        self.bn1 = nn.BatchNorm1d(n_channel)
        self.pool1 = nn.MaxPool1d(4)
        self.conv2 = nn.Conv1d(n_channel, n_channel, kernel_size=3)
        self.bn2 = nn.BatchNorm1d(n_channel)
        self.pool2 = nn.MaxPool1d(4)
        self.conv3 = nn.Conv1d(n_channel, 2 * n_channel, kernel_size=3)
        self.bn3 = nn.BatchNorm1d(2 * n_channel)
        self.pool3 = nn.MaxPool1d(4)
        self.conv4 = nn.Conv1d(2 * n_channel, 2 * n_channel, kernel_size=3)
        self.bn4 = nn.BatchNorm1d(2 * n_channel)
        self.pool4 = nn.MaxPool1d(4)
        self.fc1 = nn.Linear(2 * n_channel, n_output)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(self.bn2(x))
        x = self.pool2(x)
        x = self.conv3(x)
        x = F.relu(self.bn3(x))
        x = self.pool3(x)
        x = self.conv4(x)
        x = F.relu(self.bn4(x))
        x = self.pool4(x)
        x = F.avg_pool1d(x, x.shape[-1])
        x = x.permute(0, 2, 1)
        x = torch.squeeze(x)
        x = self.fc1(x)
        return x

In [58]:
model = M5(n_input=n_channels, n_output=len(labels))
model.to(device)
print(model)

M5(
  (conv1): Conv1d(10, 32, kernel_size=(80,), stride=(16,))
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(32, 64, kernel_size=(3,), stride=(1,))
  (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv1d(64, 64, kernel_size=(3,), stride=(1,))
  (bn4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool4): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=64, out_features=35, bias=True)
)


In [59]:
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)  # reduce the 
criterion = nn.CrossEntropyLoss()

In [63]:
def train(model, epoch, log_interval):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):

        data = data.to(device)
        target = target.to(device)

        # apply transform and model on whole batch directly on device
        output = model(data)

        # negative log-likelihood for a tensor of size (batch x 1 x n_output)
        loss = criterion(output, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print training stats
        if batch_idx % log_interval == 0:
            print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}")

        # update progress bar
        pbar.update(pbar_update)
        # record loss
        losses.append(loss.item())

In [66]:
def number_of_correct(pred, target):
    # count number of correct predictions
    return pred.squeeze().eq(target).sum().item()


def get_likely_index(tensor):
    # find most likely label index for each element in the batch
    return tensor.argmax(dim=-1)


def test(model, epoch):
    model.eval()
    correct = 0
    for data, target in val_loader:

        data = data.to(device)
        target = target.to(device)

        # apply transform and model on whole batch directly on device
        output = model(data)

        pred = get_likely_index(output)
        correct += number_of_correct(pred, target)

        # update progress bar
        pbar.update(pbar_update)

    print(f"\nTest Epoch: {epoch}\tAccuracy: {correct}/{len(val_loader.dataset)} ({100. * correct / len(val_loader.dataset):.0f}%)\n")

In [68]:
log_interval = 20
n_epoch = 2

pbar_update = 1 / (len(train_loader) + len(val_loader))
losses = []

with tqdm(total=n_epoch) as pbar:
    for epoch in range(1, n_epoch + 1):
        train(model, epoch, log_interval)
        test(model, epoch)
        scheduler.step()

  0%|                        | 0.001349527665317139/2 [00:00<05:15, 158.02s/it]

Train Epoch: 1 [0/84843 (0%)]	Loss: 1.397017


  1%|▏                       | 0.014170040485829956/2 [00:02<04:45, 143.79s/it]

Train Epoch: 1 [1280/84843 (2%)]	Loss: 1.562293


  1%|▎                       | 0.029014844804318474/2 [00:04<04:59, 151.81s/it]

...

Train Epoch: 2 [83200/84843 (98%)]	Loss: 1.378824


 95%|███████████████████████████▍ | 1.89271255060719/2 [04:06<00:09, 88.70s/it]

Train Epoch: 2 [84480/84843 (100%)]	Loss: 1.754214


100%|█████████████████████████▉| 1.9999999999998948/2 [04:12<00:00, 126.03s/it]


Test Epoch: 2	Accuracy: 6009/9981 (60%)



In [126]:
def predict(tensor):
    # Use the model to predict the label of the waveform
    tensor = tensor.to(device)
    tensor = model(tensor.unsqueeze(0))
    tensor = get_likely_index(tensor)
    tensor = index_to_label(tensor.squeeze())
    return tensor


waveform, _, sample_rate = train_set[-1]
ipd.Audio(waveform.numpy(), rate=sample_rate)

print(f"Expected: {utterance}. Predicted: {predict(waveform)}.")

Expected: 16000. Predicted: backward.


In [70]:
ig = IntegratedGradients(model)
sa = Saliency(model)
dl = DeepLift(model)
fa = FeatureAblation(model)

In [71]:
dataiter = iter(val_loader)
data, target = next(dataiter)
data = data.to(device)
target = target.to(device)

In [72]:
outputs = model(data)

In [73]:
outputs.shape

torch.Size([64, 35])

In [74]:
preds = outputs.argmax(dim=-1)

In [75]:
preds.shape

torch.Size([64])

## SA

In [76]:
sa_attr = sa.attribute(data, preds)

In [78]:
sa_attr.shape

torch.Size([64, 10, 16000])

In [95]:
avg_sa_attr = sa_attr.abs().mean(-1).mean(0).detach().cpu().numpy()

In [118]:
avg_sa_attr/np.linalg.norm(avg_sa_attr, 1)

array([0.6493612 , 0.03742831, 0.04198129, 0.03836196, 0.03845372,
       0.0374977 , 0.04018834, 0.0392105 , 0.0384088 , 0.03910828],
      dtype=float32)

## DL

In [97]:
dl_attr = dl.attribute(data, torch.zeros_like(data).to(device), preds)

In [98]:
dl_attr.shape

torch.Size([64, 10, 16000])

In [99]:
avg_dl_attr = dl_attr.abs().mean(-1).mean(0).detach().cpu().numpy()

In [119]:
avg_dl_attr/np.linalg.norm(avg_dl_attr, 1)

array([0.30607808, 0.07396897, 0.08260281, 0.07592342, 0.07581228,
       0.07408911, 0.07898825, 0.07813543, 0.07642604, 0.07797565],
      dtype=float32)

## IG

In [106]:
ig_attr = ig.attribute(data, torch.zeros_like(data).to(device), preds)

In [107]:
ig_attr.shape

torch.Size([64, 10, 16000])

In [108]:
avg_ig_attr = ig_attr.abs().mean(-1).mean(0).detach().cpu().numpy()

In [120]:
avg_ig_attr/np.linalg.norm(avg_ig_attr, 1)

array([0.38590728, 0.06523575, 0.07424971, 0.0675805 , 0.06741073,
       0.06510134, 0.07118871, 0.06848693, 0.06662644, 0.06821263])

## FA

In [121]:
feature_mask = np.arange(n_channels)

In [135]:
feature_mask = feature_mask[np.newaxis,:,np.newaxis]

In [136]:
feature_mask = feature_mask.repeat(sample_rate, axis=-1).repeat(batch_size, axis=0)

In [139]:
feature_mask=torch.from_numpy(feature_mask)

In [140]:
fa_attr = fa.attribute(data, torch.zeros_like(data).to(device), target=preds, feature_mask=feature_mask.to(device))

In [141]:
fa_attr.shape

torch.Size([64, 10, 16000])

In [142]:
avg_fa_attr = fa_attr.abs().mean(-1).mean(0).detach().cpu().numpy()

In [ ]:
avg_fa_attr/np.linalg.norm(avg_fa_attr, 1)